<a href="https://colab.research.google.com/github/Rana-ob/learn--for-GP/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# تثبيت مكتبات معالجة النماذج والواجهة
!pip install transformers accelerate bitsandbytes gradio

# تثبيت PyTorch (غالباً يكون موجود، للتأكد ندعم CUDA 11.8)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gradio as gr

# اسم النموذج اللي بنستخدمه
# Gemma 7B Instruct يعتبر ممتاز ومناسب لـ Colab GPU
model_id = "google/gemma-2b-it"

# إعدادات Quantization (تقليل استهلاك الذاكرة عشان يشتغل على GPU Colab)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# تحميل الـ Tokenizer والنموذج
print(f"جاري تحميل النموذج {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" # بيحمل النموذج على الـ GPU تلقائياً
)
print(f"تم تحميل النموذج على الجهاز: {model.device}")
print("جاهز للبدء!")

جاري تحميل النموذج google/gemma-7b-it...


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-7b-it.
403 Client Error. (Request ID: Root=1-688c5aa1-7d4a008566ff0c441af71061;f5008c4f-276c-4e20-9608-97ec7d690422)

Cannot access gated repo for url https://huggingface.co/google/gemma-7b-it/resolve/main/config.json.
Your request to access model google/gemma-7b-it is awaiting a review from the repo authors.